In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import sparse
from scipy.sparse.linalg import spsolve
import scipy.integrate as integrate
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
# Import previously generated data
try:
    df = pd.read_csv('synthetic_radar_data.csv')
    print("Loaded synthetic data:")
    print(df.head())
except FileNotFoundError:
    print("Synthetic data file not found. Run Notebook 1 first.")
    # Create a simplified version for demonstration
    from sklearn.datasets import make_classification
    X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=2, random_state=42)
    df = pd.DataFrame(X, columns=['reflectivity_dbz', 'differential_reflectivity_db', 
                                 'specific_differential_phase_deg_km', 'spectrum_width', 'velocity'])
    df['is_hail'] = y
    df['particle_type'] = ['hail' if val == 1 else 'rain' for val in y]
    df['diameter_mm'] = np.random.uniform(0.5, 30, 1000)
    df['aspect_ratio'] = np.random.uniform(0.8, 1.2, 1000)
    df['azimuth'] = np.random.uniform(0, 360, 1000)
    df['range_km'] = np.random.uniform(10, 100, 1000)
    print("Created synthetic data for demonstration.")

Synthetic data file not found. Run Notebook 1 first.
Created synthetic data for demonstration.


In [4]:
### importing real data
dfreal = pd.read_csv(r"D:\Documents\UNAI_Notes\HailStorms\data\processed\cleaned_nexrad_data3.csv")

In [5]:
dfreal

,Azimuth,Range,differential_reflectivity,specific_differential_phase,spectrum_width,velocity
0,3.0,1998.00,1.1875,0.10,0.0,4.5
1,3.0,2247.75,-0.7500,0.10,0.0,5.0
2,3.0,2497.50,-0.3750,0.10,0.0,5.0
3,3.0,3996.00,-1.0625,0.10,8.0,-10.0
4,3.0,7742.25,-0.1250,0.10,8.0,-2.0
...,...,...,...,...,...,...
10371,359.0,136863.00,0.0625,0.00,0.0,10.0
10372,359.0,145854.00,-0.6875,0.05,0.0,10.0
10373,359.0,146853.00,-0.3125,0.00,4.0,0.0
10374,359.0,144855.00,-0.1250,0.05,4.0,10.0


In [ ]:
df  ## synthetic data

,reflectivity_dbz,differential_reflectivity_db,specific_differential_phase_deg_km,spectrum_width,velocity,is_hail,particle_type,diameter_mm,aspect_ratio,azimuth,range_km
0,-0.065300,-0.717214,0.393952,-0.934473,1.681514,0,rain,10.588112,0.871564,298.169469,93.805013
1,0.567015,-0.044606,1.612851,-1.350174,2.488878,0,rain,21.897821,0.912149,298.391556,69.319507
2,-0.247215,-0.650569,-0.743500,-1.214190,0.841110,0,rain,27.945065,1.044563,44.311373,30.425482
3,1.145870,0.974224,1.562506,-2.277010,2.276521,1,hail,17.146376,0.991225,27.582395,49.863474
4,0.599605,-0.427545,2.374472,-1.503510,3.604959,0,rain,28.660431,1.131772,24.308395,38.421148
...,...,...,...,...,...,...,...,...,...,...,...
995,0.286381,-0.448468,0.690227,-1.963318,2.498798,1,hail,10.078362,1.028377,81.307771,11.394729
996,-0.188744,-1.292454,0.944265,-0.931308,2.658180,0,rain,12.404203,1.149630,185.657179,81.755743
997,0.883609,-0.009022,1.507666,-3.324101,3.880482,1,hail,26.236889,1.174618,308.669932,17.550274
998,0.045412,-0.169270,-1.100362,-2.224787,0.888160,0,rain,13.410271,0.969035,70.451087,86.549660


In [8]:
# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


# Physical constants
c = 3e8  # Speed of light in vacuum (m/s)
epsilon_0 = 8.85e-12  # Vacuum permittivity (F/m)
mu_0 = 1.257e-6  # Vacuum permeability (H/m)

# Radar Parameters
radar_freq = 2.8e9  # Radar frequency (Hz)
radar_wavelength = c / radar_freq  # Radar wavelength (m)
radar_wavenumber = 2 * np.pi / radar_wavelength  # Radar wavenumber (1/m)

# 1. FEM Mesh Generation


In [19]:
def generate_2d_mesh(domain_size, n_elements):
    """
    Generate a 2D triangular mesh for FEM Simulation

    Args:
        domain_size: Size of square domain (m)
        n_elements: Number of elements along each axis
    """
    
    x = np.linspace(0, domain_size, n_elements + 1)
    y = np.linspace(0, domain_size, n_elements + 1)
    xx, yy = np.meshgrid(x, y)
    
    nodes = np.vstack((xx.flatten(), yy.flatten())).T
    
    ## create a tringular elements
    elements = []
    
    for i in range(n_elements):
        for j in range(n_elements):
            ## node indices
            n1 = i * (n_elements + 1) + j
            n2 = (i + 1) * (n_elements + 1) + j
            n3 = (i + 1) * (n_elements + 1) + (j + 1)
            n4 = i * (n_elements + 1) + (j + 1)
            
            # Two triangles per square
            elements.append([n1, n2, n3])
            elements.append([n1, n3, n4])
            
    return nodes, np.array(elements)


def assemble_fem_metrics(nodes, elements, k_wave, epsilon_r):
    """
    Assemble FEM matrices for Helmholtz equation.
    
    Args:
        nodes: Node coordinates
        elements: Element connectivity
        k_wave: Wave number
        epsilon_r: Relative permittivity function
        
    Returns:
        Tuple of (A, b) where A is system matrix and b is RHS
    """
    n_nodes = nodes.shape[0]
    n_elements = elements.shape[0]
    
    # Initialize sparse matrix
    A = sparse.lil_matrix((n_nodes, n_nodes), dtype=complex)
    b = np.zeros(n_nodes, dtype=complex)
    
    # Loop over elements
    for el in elements:
        # Element nodes
        n1, n2, n3 = el
        
        # Node coordinates
        x1, y1 = nodes[n1]
        x2, y2 = nodes[n2]
        x3, y3 = nodes[n3]
        
        # Element area
        area = 0.5 * np.abs((x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1))
        
        # Element center for material properties
        x_center = (x1 + x2 + x3) / 3
        y_center = (y1 + y2 + y3) / 3
        
        # Get permittivity at element center
        eps_r = epsilon_r(x_center, y_center)
        
        # Shape function gradients
        b1 = (y2 - y3) / (2 * area)
        b2 = (y3 - y1) / (2 * area)
        b3 = (y1 - y2) / (2 * area)
        
        c1 = (x3 - x2) / (2 * area)
        c2 = (x1 - x3) / (2 * area)
        c3 = (x2 - x1) / (2 * area)
        
        # Element matrix
        K_el = np.zeros((3, 3), dtype=complex)
        M_el = np.zeros((3, 3), dtype=complex)
        
        # Shape function indices
        indices = [0, 1, 2]
        
        # Assemble local stiffness and mass matrices
        for i in indices:
            for j in indices:
                # Stiffness term: ∫(∇φᵢ·∇φⱼ) dA
                K_el[i, j] = (b1*b1 + c1*c1) * area if i == 0 and j == 0 else \
                             (b2*b2 + c2*c2) * area if i == 1 and j == 1 else \
                             (b3*b3 + c3*c3) * area if i == 2 and j == 2 else \
                             (b1*b2 + c1*c2) * area if (i == 0 and j == 1) or (i == 1 and j == 0) else \
                             (b1*b3 + c1*c3) * area if (i == 0 and j == 2) or (i == 2 and j == 0) else \
                             (b2*b3 + c2*c3) * area
                
                # Mass term: ∫(φᵢ·φⱼ) dA
                if i == j:
                    M_el[i, j] = area / 6  # Diagonal terms
                else:
                    M_el[i, j] = area / 12  # Off-diagonal terms
        
        # Local to global assembly
        nodes_el = [n1, n2, n3]
        
        for i in range(3):
            for j in range(3):
                # A = K - k²·ε_r·M
                A[nodes_el[i], nodes_el[j]] += K_el[i, j] - (k_wave**2) * eps_r * M_el[i, j]
    
    # Convert to CSR format for efficient solver
    A = A.tocsr()
    
    return A, b

In [ ]:
def apply_source_and_bc(A, b, nodes, source_pos, source_amplitude):
    """
    Apply source term and boundary conditions.
    
    Args:
        A: System matrix
        b: RHS vector
        nodes: Node coordinates
        source_pos: Position of the source (x, y)
        source_amplitude: Amplitude of the source
        
    Returns:
        Modified A and b
    """
    n_nodes = nodes.shape[0]
    # Find closest node to source position
    source_x, source_y = source_pos
    distances = np.sqrt((nodes[:, 0] - source_x)**2 + (nodes[:, 1] - source_y)**2)
    source_node = np.argmin(distances)
    
    # Apply source (as a point source)
    b[source_node] = source_amplitude
    
    # Apply Dirichlet boundary conditions (set field to zero at boundaries)
    # Find boundary nodes (nodes at the domain edges)
    x_min, y_min = np.min(nodes, axis=0)
    x_max, y_max = np.max(nodes, axis=0)
    eps = 1e-6  # Small tolerance
    
    boundary_nodes = []
    for i in range(n_nodes):
        x, y = nodes[i]
        if (abs(x - x_min) < eps or abs(x - x_max) < eps or 
            abs(y - y_min) < eps or abs(y - y_max) < eps):
            boundary_nodes.append(i)
    
    # Apply Dirichlet BC (E = 0 at boundaries)
    for node in boundary_nodes:
        # Zero out row and column
        A[node, :] = 0
        A[:, node] = 0
        A[node, node] = 1
        b[node] = 0
    
    return A, b

# 4. Define Material Properties
def create_permittivity_function(particles, background_permittivity):
    """
    Create a function that returns the permittivity at any point.
    
    Args:
        particles: List of tuples [(x, y, radius, permittivity), ...]
        background_permittivity: Permittivity of the background medium
        
    Returns:
        Function that takes (x, y) and returns permittivity
    """
    def permittivity_function(x, y):
        # Default to background permittivity
        result = background_permittivity
        
        # Check if point is inside any particle
        for px, py, radius, perm in particles:
            distance = np.sqrt((x - px)**2 + (y - py)**2)
            if distance <= radius:
                result = perm
                break
        
        return result
    
    return permittivity_function

# 5. Solve FEM System
def solve_fem_system(A, b):
    """
    Solve the FEM system using sparse solver.
    
    Args:
        A: System matrix
        b: RHS vector
        
    Returns:
        Solution vector
    """
    # Solve the system Ax = b
    try:
        x = spsolve(A, b)
        return x
    except Exception as e:
        print(f"Error solving system: {e}")
        return np.zeros_like(b)

# 6. Calculate Radar Backscatter from FEM Solution
def calculate_backscatter(solution, nodes, elements, k_wave, incident_direction):
    """
    Calculate radar backscatter from FEM solution.
    
    Args:
        solution: FEM solution vector
        nodes: Node coordinates
        elements: Element connectivity
        k_wave: Wave number
        incident_direction: Direction of incident wave (theta in radians)
        
    Returns:
        Backscatter cross-section
    """
    # Direction vectors
    incident_vec = np.array([np.cos(incident_direction), np.sin(incident_direction)])
    backscatter_vec = -incident_vec  # Directly back
    
    # Domain center
    domain_center = np.mean(nodes, axis=0)
    
    # Compute far-field integral over all elements
    far_field = 0.0
    
    for el in elements:
        # Element nodes
        n1, n2, n3 = el
        
        # Node coordinates
        x1, y1 = nodes[n1]
        x2, y2 = nodes[n2]
        x3, y3 = nodes[n3]
        
        # Element area
        area = 0.5 * np.abs((x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1))
        
        # Element center
        x_center = (x1 + x2 + x3) / 3
        y_center = (y1 + y2 + y3) / 3
        center = np.array([x_center, y_center])
        
        # Field value at element center (approximate using shape functions)
        field_value = (solution[n1] + solution[n2] + solution[n3]) / 3
        
        # Phase factor e^{-jk r̂_s·r'}
        phase = np.exp(-1j * k_wave * np.dot(backscatter_vec, center - domain_center))
        
        # Add contribution to far-field
        far_field += field_value * phase * area
    
    # Backscatter cross-section (simplified)
    rcs = 4 * np.pi * np.abs(far_field)**2
    
    return rcs

# 7. Implement Adaptive Sampling
def residual_based_adaptive_refinement(nodes, elements, solution, k_wave, eps_r, error_threshold=0.01):
    """
    Implement residual-based adaptive refinement.
    
    Args:
        nodes: Node coordinates
        elements: Element connectivity
        solution: Current FEM solution
        k_wave: Wave number
        eps_r: Permittivity function
        error_threshold: Threshold for refinement
        
    Returns:
        Tuple of (refined_nodes, refined_elements, element_errors)
    """
    # Calculate element errors
    element_errors = np.zeros(len(elements))
    
    for i, el in enumerate(elements):
        # Element nodes
        n1, n2, n3 = el
        
        # Node coordinates
        x1, y1 = nodes[n1]
        x2, y2 = nodes[n2]
        x3, y3 = nodes[n3]
        
        # Element center
        x_center = (x1 + x2 + x3) / 3
        y_center = (y1 + y2 + y3) / 3
        
        # Element area
        area = 0.5 * np.abs((x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1))
        
        # Field values at nodes
        u1 = solution[n1]
        u2 = solution[n2]
        u3 = solution[n3]
        
        # Permittivity at center
        epsilon = eps_r(x_center, y_center)
        
        # Shape function gradients
        b1 = (y2 - y3) / (2 * area)
        b2 = (y3 - y1) / (2 * area)
        b3 = (y1 - y2) / (2 * area)
        
        c1 = (x3 - x2) / (2 * area)
        c2 = (x1 - x3) / (2 * area)
        c3 = (x2 - x1) / (2 * area)
        
        # Calculate residual (simplified)
        grad_u_x = u1 * b1 + u2 * b2 + u3 * b3
        grad_u_y = u1 * c1 + u2 * c2 + u3 * c3
        laplacian_u = 0  # Approximate
        
        # Helmholtz residual: ∇²u + k²εᵣu
        residual = laplacian_u + k_wave**2 * epsilon * (u1 + u2 + u3) / 3
        
        # Error estimate
        element_errors[i] = np.abs(residual) * area
    
    # Identify elements to refine (above threshold)
    elements_to_refine = np.where(element_errors > error_threshold * np.max(element_errors))[0]
    
    # Placeholder for actual refinement
    # In a real implementation, we would subdivide these elements
    # For this demonstration, we'll just highlight the elements that would be refined
    
    return nodes, elements, element_errors, elements_to_refine

# 8. Implement Monte Carlo Simulation
def monte_carlo_simulation(n_samples, domain_size, particles_range, wavelength):
    """
    Run Monte Carlo simulation to analyze uncertainty.
    
    Args:
        n_samples: Number of Monte Carlo samples
        domain_size: Size of simulation domain
        particles_range: Range of particle parameters [(min_radius, max_radius), (min_perm_real, max_perm_real), ...]
        wavelength: Radar wavelength
        
    Returns:
        Array of backscatter cross-sections from all simulations
    """
    k_wave = 2 * np.pi / wavelength
    
    # Results array
    results = np.zeros(n_samples, dtype=complex)
    
    # Parameter ranges
    min_radius, max_radius = particles_range[0]
    min_perm_real, max_perm_real = particles_range[1]
    min_perm_imag, max_perm_imag = particles_range[2]
    
    for i in range(n_samples):
        # Generate random particle
        radius = np.random.uniform(min_radius, max_radius)
        perm_real = np.random.uniform(min_perm_real, max_perm_real)
        perm_imag = np.random.uniform(min_perm_imag, max_perm_imag)
        perm = complex(perm_real, -perm_imag)
        
        # Place particle in center of domain
        particles = [(domain_size/2, domain_size/2, radius, perm)]
        
        # Define permittivity function
        eps_r = create_permittivity_function(particles, 1.0)
        
        # Generate mesh (simplified with fewer elements for speed)
        n_elements = 20  # Lower resolution for Monte Carlo
        nodes, elements = generate_2d_mesh(domain_size, n_elements)
        
        # Assemble system
        A, b = assemble_fem_matrices(nodes, elements, k_wave, eps_r)
        
        # Apply source and boundary conditions
        source_pos = (0.1 * domain_size, 0.5 * domain_size)  # Source near left edge
        A, b = apply_source_and_bc(A, b, nodes, source_pos, 1.0)
        
        # Solve system
        solution = solve_fem_system(A, b)
        
        # Calculate backscatter
        incident_direction = 0  # From left to right
        rcs = calculate_backscatter(solution, nodes, elements, k_wave, incident_direction)
        
        # Store result
        results[i] =# Notebook 2: FEM-Based Scattering Simulation and Precipitation Analysis
